In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import itertools
from utils import COLORS, color_coding
from pylab import rcParams

In [2]:
GRAPH_PATH_YO = './graphs/yo_2.graphml'
GRAPH_PATH_OTRO = './graphs/otro_2.graphml'
SOCIODEMOGRAPHIC_PATH = "./data/Sociodemographic_survey/answers_sociodemographic_survey.pkl"

In [3]:
G_yo = nx.read_graphml(GRAPH_PATH_YO)
G_otro = nx.read_graphml(GRAPH_PATH_OTRO)

In [4]:
adj = G_yo.adjacency()
Gnodes = G_yo.nodes

In [5]:
for a, b in adj:
    # print(f'a: {a}\nb: {b}')
    break

In [6]:
for b1, b2 in b.items():
    print(b1, b2['weight'])
    break

bd4b2f4 0.45098039215686264


In [7]:
def node_attribute_xy(G, attribute, nodes=None):
    """Returns iterator of node-attribute pairs for all edges in G, with the weight of the edge connecting them.

    Parameters
    ----------
    G: NetworkX graph

    attribute: key
       The node attribute key.

    nodes: list or iterable (optional)
        Use only edges that are incident to specified nodes.
        The default is all nodes.

    Returns
    -------
    (x, y, z): 2-tuple
        Generates 2-tuple of (attribute, attribute, weight) values.

    Examples
    --------
    >>> G = nx.DiGraph()
    >>> G.add_node(1, color="red")
    >>> G.add_node(2, color="blue")
    >>> G.add_edge(1, 2)
    >>> weight = 0.1
    >>> G.add_weighted_edges_from(list((1, n, weight) for n in G.nodes))
    >>> list(nx.node_attribute_xy(G, "color"))
    [('red', 'blue', 0.1)]

    Notes
    -----
    For undirected graphs each edge is produced twice, once for each edge
    representation (u, v) and (v, u), with the exception of self-loop edges
    which only appear once.
    """
    if nodes is None:
        nodes = set(G)
    else:
        nodes = set(nodes)
    Gnodes = G.nodes
    for u, nbrsdict in G.adjacency():
        if u not in nodes:
            continue
        uattr = Gnodes[u].get(attribute, None)
        if G.is_multigraph():
            for v, keys in nbrsdict.items():
                vattr = Gnodes[v].get(attribute, None)
                for _ in keys:
                    yield (uattr, vattr)
        else:
            for v, w in nbrsdict.items():
                vattr = Gnodes[v].get(attribute, None)
                uvweight = w['weight']
                yield (uattr, vattr, uvweight)

In [45]:
def mixing_dict(xyz, normalized=False):
    """Returns a dictionary representation of mixing matrix.

    Parameters
    ----------
    xyz : list or container of three-tuples
       Pairs of (x,y,z) items (attribute, attribute, weight).

    attribute : string
       Node attribute key

    normalized : bool (default=False)
       Return counts if False or probabilities if True.

    Returns
    -------
    d: dictionary
       Counts or Joint probability of occurrence of values in xy.
    """
    d = {}
    psum = 0.0
    for x, y, z in xyz:
        if x not in d:
            d[x] = {}
        if y not in d:
            d[y] = {}
        v = d[x].get(y, 0)
        d[x][y] = v + z
        psum += z
        # print(psum)

    if normalized:
        for _, jdict in d.items():
            for j in jdict:
                jdict[j] /= psum
    print(f'psum: {psum}')
    return d

In [9]:
iter_xyz = node_attribute_xy(G_yo, 'age_bracket')
mix_dict = mixing_dict(iter_xyz, normalized=True)
mix_dict

{'old': {'young': 0.2390580614811188, 'old': 0.3860572834681267},
 'young': {'old': 0.23905806148111858, 'young': 0.1358265935696318}}

In [10]:
def attribute_mixing_dict(G, attribute, nodes=None, normalized=False):
    """Returns dictionary representation of mixing matrix for attribute.

    Parameters
    ----------
    G : graph
       NetworkX graph object.

    attribute : string
       Node attribute key.

    nodes: list or iterable (optional)
        Unse nodes in container to build the dict. The default is all nodes.

    normalized : bool (default=False)
       Return counts if False or probabilities if True.

    Examples
    --------
    >>> G = nx.Graph()
    >>> G.add_nodes_from([0, 1], color="red")
    >>> G.add_nodes_from([2, 3], color="blue")
    >>> G.add_edge(1, 3)
    >>> d = nx.attribute_mixing_dict(G, "color")
    >>> print(d["red"]["blue"])
    1
    >>> print(d["blue"]["red"])  # d symmetric for undirected graphs
    1

    Returns
    -------
    d : dictionary
       Counts or joint probability of occurrence of attribute pairs.
    """
    xy_iter = node_attribute_xy(G, attribute, nodes)
    return mixing_dict(xy_iter, normalized=normalized)

In [11]:
mix_dict_2 = attribute_mixing_dict(G_yo, 'age_bracket', normalized=True)
mix_dict_2

{'old': {'young': 0.2390580614811188, 'old': 0.3860572834681267},
 'young': {'old': 0.23905806148111858, 'young': 0.1358265935696318}}

In [12]:
from networkx.utils import dict_to_numpy_array

In [13]:
def attribute_mixing_matrix(G, attribute, nodes=None, mapping=None, normalized=True):
    """Returns mixing matrix for attribute.

    Parameters
    ----------
    G : graph
       NetworkX graph object.

    attribute : string
       Node attribute key.

    nodes: list or iterable (optional)
        Use only nodes in container to build the matrix. The default is
        all nodes.

    mapping : dictionary, optional
       Mapping from node attribute to integer index in matrix.
       If not specified, an arbitrary ordering will be used.

    normalized : bool (default=True)
       Return counts if False or probabilities if True.

    Returns
    -------
    m: numpy array
       Counts or joint probability of occurrence of attribute pairs.

    Notes
    -----
    If each node has a unique attribute value, the unnormalized mixing matrix
    will be equal to the adjacency matrix. To get a denser mixing matrix,
    the rounding can be performed to form groups of nodes with equal values.
    For example, the exact height of persons in cm (180.79155222, 163.9080892,
    163.30095355, 167.99016217, 168.21590163, ...) can be rounded to (180, 163,
    163, 168, 168, ...).

    Definitions of attribute mixing matrix vary on whether the matrix
    should include rows for attribute values that don't arise. Here we
    do not include such empty-rows. But you can force them to appear
    by inputting a `mapping` that includes those values.

    Examples
    --------
    >>> G = nx.path_graph(3)
    >>> gender = {0: 'male', 1: 'female', 2: 'female'}
    >>> nx.set_node_attributes(G, gender, 'gender')
    >>> mapping = {'male': 0, 'female': 1}
    >>> mix_mat = nx.attribute_mixing_matrix(G, 'gender', mapping=mapping)
    >>> # mixing from male nodes to female nodes
    >>> mix_mat[mapping['male'], mapping['female']]
    0.25
    """
    d = attribute_mixing_dict(G, attribute, nodes)
    a = dict_to_numpy_array(d, mapping=mapping)
    if normalized:
        a = a / a.sum()
    return a


In [14]:
attr_mat = attribute_mixing_matrix(G_yo, 'age_bracket')
attr_mat

array([[0.38605728, 0.23905806],
       [0.23905806, 0.13582659]])

In [15]:
attr_mat.sum()

1.0

In [16]:
def attribute_ac(M):
    """Compute assortativity for attribute matrix M.

    Parameters
    ----------
    M : numpy.ndarray
        2D ndarray representing the attribute mixing matrix.

    Notes
    -----
    This computes Eq. (2) in Ref. [1]_ , (trace(e)-sum(e^2))/(1-sum(e^2)),
    where e is the joint probability distribution (mixing matrix)
    of the specified attribute.

    References
    ----------
    .. [1] M. E. J. Newman, Mixing patterns in networks,
       Physical Review E, 67 026126, 2003
    """
    if M.sum() != 1.0:
        M = M / M.sum()
    s = (M @ M).sum()
    t = M.trace()
    r = (t - s) / (1 - s)
    r_min = (-s) / (1 - s)
    print(r_min)
    return r

In [17]:
def attribute_assortativity_coefficient(G, attribute, nodes=None):
    """Compute assortativity for node attributes.

    Assortativity measures the similarity of connections
    in the graph with respect to the given attribute.

    Parameters
    ----------
    G : NetworkX graph

    attribute : string
        Node attribute key

    nodes: list or iterable (optional)
        Compute attribute assortativity for nodes in container.
        The default is all nodes.

    Returns
    -------
    r: float
       Assortativity of graph for given attribute

    Examples
    --------
    >>> G = nx.Graph()
    >>> G.add_nodes_from([0, 1], color="red")
    >>> G.add_nodes_from([2, 3], color="blue")
    >>> G.add_edges_from([(0, 1), (2, 3)])
    >>> print(nx.attribute_assortativity_coefficient(G, "color"))
    1.0

    Notes
    -----
    This computes Eq. (2) in Ref. [1]_ , (trace(M)-sum(M^2))/(1-sum(M^2)),
    where M is the joint probability distribution (mixing matrix)
    of the specified attribute.

    References
    ----------
    .. [1] M. E. J. Newman, Mixing patterns in networks,
       Physical Review E, 67 026126, 2003
    """
    M = attribute_mixing_matrix(G, attribute, nodes)
    return attribute_ac(M)

In [18]:
r = attribute_assortativity_coefficient(G_yo, 'age_bracket')
r

-1.1335959606006796


-0.020106628650291496

In [19]:
r_otro = attribute_assortativity_coefficient(G_otro, 'age_bracket')
r_otro

-1.2372737673378285


-0.02133459053812851

## Test with small graph

Load graph with only 4 nodes to test computation manually

In [33]:
GRAPH_PATH_YO_4_NODES = './graphs/yo_3.graphml'

In [34]:
G4 = nx.read_graphml(GRAPH_PATH_YO_4_NODES)

In [36]:
r4 = attribute_assortativity_coefficient(G4, 'age_bracket')
r4

-1.462380300957593


-0.39534883720930275

In [40]:
edges4 = nx.get_edge_attributes(G4, 'weight')

In [41]:
edges4

{('4e53c5c', 'bd4b2f4'): 0.45098039215686264,
 ('4e53c5c', 'aa07d95'): 0.36274509803921573,
 ('4e53c5c', 'db7d9c3'): 0.19117647058823528,
 ('bd4b2f4', 'aa07d95'): 0.35784313725490197,
 ('bd4b2f4', 'db7d9c3'): 0.19117647058823528,
 ('aa07d95', 'db7d9c3'): 0.21078431372549017}

In [42]:
attributes4 = nx.get_node_attributes(G4, 'age_bracket')
attributes4

{'4e53c5c': 'old', 'bd4b2f4': 'young', 'aa07d95': 'old', 'db7d9c3': 'old'}

In [49]:
iter_xyz = node_attribute_xy(G4, 'age_bracket')
for x, y, z in iter_xyz:
    print(x, y, z)

old young 0.45098039215686264
old old 0.36274509803921573
old old 0.19117647058823528
young old 0.45098039215686264
young old 0.35784313725490197
young old 0.19117647058823528
old old 0.36274509803921573
old young 0.35784313725490197
old old 0.21078431372549017
old old 0.19117647058823528
old young 0.19117647058823528
old old 0.21078431372549017


In [48]:
mix_dict = mixing_dict(iter_xyz, normalized=True)
mix_dict

psum: 3.529411764705883


{'old': {'young': 0.2833333333333332, 'old': 0.4333333333333332},
 'young': {'old': 0.2833333333333332}}

In [50]:
attr_mat = attribute_mixing_matrix(G4, 'age_bracket')
attr_mat

psum: 3.529411764705883


array([[0.43333333, 0.28333333],
       [0.28333333, 0.        ]])